# Skeleton Code for the Final Project

Github repo link: https://github.com/GSDSAML/prompt-engineering-2024fall/tree/main



## Leaderboard

Leaderboard updated from the Github repo.

In [ ]:
!rm -f leaderboard.csv
!wget https://raw.githubusercontent.com/GSDSAML/prompt-engineering-2024fall/refs/heads/main/final_project/leaderboard.csv

In [ ]:
import pandas as pd

# Load the leaderboard.csv file into a pandas DataFrame.
df = pd.read_csv('leaderboard.csv')

df

## Data preparation

### You can download the FinQA train and test set here

In [ ]:
!wget https://github.com/czyssrs/FinQA/raw/refs/heads/main/dataset/train.json

In [ ]:
!wget https://github.com/czyssrs/FinQA/raw/refs/heads/main/dataset/test.json

## Environment Setup

In [ ]:
# Install langchain
!pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

### Set API key

In [ ]:
# Set API key
OPENAI_API_KEY="your_api_key_here"

### Prepare model

In [ ]:
# Prepare model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, api_key=OPENAI_API_KEY)

## Data Preparation

### Download and unzip backup vector DB

*   Source dataset: [FinQA](https://finqasite.github.io/)
*   EDA for the FinQA dataset: https://medium.com/@sohilsharma1996/finqa-project-numerical-reasoning-over-financial-data-77a3fb4133d2

You can build your own vector DB for retrieval.

In [ ]:
!wget https://github.com/GSDSAML/prompt-engineering-2024fall/raw/refs/heads/main/final_project/db.zip

In [ ]:
!unzip db.zip

### Fiscal year end date dictionary

Collected from SEC EDGAR

Dictionary with Key(ticker): Value(Fiscal year end date)

Actually, the date may different every year, but we will use this for now.

In [ ]:
company_fiscal_year_end = {'GIS': '0530', 'FRT': '1231', 'MMM': '1231', 'K': '1228', 'INTC': '1228', 'CAG': '0525', 'MSI': '1231', 'FIS': '1231', 'CCI': '1231', 'CDW': '1231', 'GPN': '1231', 'TMUS': '1231', 'D': '1231', 'KMI': '1231', 'NTAP': '0426', 'TSCO': '1230', 'BLK': '1231', 'CNC': '1231', 'MRK': '1231', 'JKHY': '0630', 'VTR': '1231', 'GRMN': '1228', 'AMT': '1231', 'MAR': '1231', 'HIG': '1231', 'ADI': '1102', 'ILMN': '1229', 'NKE': '0531', 'VNO': '1231', 'FITB': '1231', 'ETR': '1231', 'KIM': '1231', 'AMAT': '1027', 'OKE': '1231', 'DG': '0131', 'TFX': '1231', 'AAL': '1231', 'EXPD': '1231', 'V': '0930', 'REGN': '1231', 'PPG': '1231', 'RCL': '1231', 'VLO': '1231', 'KHC': '1228', 'HUM': '1231', 'WELL': '1231', 'ALLE': '1231', 'UNP': '1231', 'LLY': '1231', 'TXN': '1231', 'GS': '1231', 'LKQ': '1231', 'SYY': '0629', 'IPG': '1231', 'AAP': '1228', 'ECL': '1231', 'MKTX': '1231', 'C': '1231', 'ZBH': '1231', 'UAA': '0331', 'MRO': '1231', 'AON': '1231', 'SPGI': '1231', 'ORLY': '1231', 'MO': '1231', 'PM': '1231', 'BKNG': '1231', 'BKR': '1231', 'AWK': '1231', 'APTV': '1231', 'WRK': '0930', 'SWKS': '0927', 'CMCSA': '1231', 'CE': '1231', 'CB': '1231', 'WMT': '0131', 'STT': '1231', 'RSG': '1231', 'AOS': '1231', 'AAPL': '0928', 'EXR': '1231', 'UA': '0331', 'CME': '1231', 'HII': '1231', 'HOLX': '0928', 'SLB': '1231', 'EW': '1231', 'EMR': '0930', 'VRTX': '1231', 'APD': '0930', 'ADBE': '1129', 'CAT': '1231', 'PNC': '1231', 'MAA': '1231', 'AES': '1231', 'SNA': '1228', 'JPM': '1231', 'NCLH': '1231', 'PKG': '1231', 'TROW': '1231', 'HST': '1231', 'NWS': '0630', 'MA': '1231', 'DVN': '1231', 'ANSS': '1231', 'ANET': '1231', 'SNPS': '1031', 'RL': '0328', 'DISH': '1231', 'HWM': '1231', 'FTV': '1231', 'L': '1231', 'LMT': '1231', 'MAS': '1231', 'STZ': '0228', 'EMN': '1231', 'CDNS': '1231', 'EL': '0630', 'SLG': '1231', 'BDX': '0930', 'UPS': '1231', 'MS': '1231', 'IQV': '1231', 'IP': '1231', 'EOG': '1231'}

## Define Tools

### LangChain Tool Use Example

List of pre-defined tools in LangChain: https://python.langchain.com/docs/integrations/tools/

In [ ]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

@tool
def add(a: int, b: int) -> int:
    """Add two integers."""
    return a + b

print(add.name)
print(add.description)
print(add.args)

In [ ]:
tools = [add, multiply]

In [ ]:
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query).tool_calls

In [ ]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

In [ ]:
llm_with_tools.invoke(messages).content

### Extract fiscal tool

In [ ]:
company_fiscal_year_end

Extract company ticker from the question

We used LLM for now, but may need to connect DB for stability.

In [ ]:
from langchain_core.prompts import PromptTemplate

extract_ticker_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
    You have to extract the company name in the given question, and answer the ticker of the company.
    Answer in JSON format, with 'ticker' key.

    Question: {question}

    Output JSON: {{
      "ticker": "<ticker of the company>"
    }}
    """
)

from pydantic import BaseModel, Field

class Ticker(BaseModel):
    """Ticker of the company in the given question"""
    ticker: str = Field(description="ticker of the company")

extract_ticker_chain = extract_ticker_prompt | llm.with_structured_output(Ticker)

In [ ]:
extract_ticker_chain.invoke({"question": "What was the revenue of Apple in 2022?"})

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime

parser = StrOutputParser()

change_prompt = PromptTemplate(
    input_variables=["original_question", "fiscal_info", "today"],
    template="""
    You have to convert relative time information in the given original question to the fiscal year of the company.
    Use the fiscal end date to convert to fiscal year.
    Answer only the converted question.

    Original question: {original_question}
    Fiscal end date for the company: {fiscal_info}
    Today date: {today}
    """
)

convert_chain = change_prompt | llm | parser

def get_fiscal_info_with_ticker(ticker: str) -> str:
    """Get fiscal year information of the company with ticker string."""
    ticker = ticker.upper()
    if ticker in company_fiscal_year_end:
      end_date = company_fiscal_year_end[ticker]
      return end_date[:2] + '-' + end_date[2:]
    else:
      return '12-31'

extract_fiscal_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
    You have to extract the fiscal year in the given question.
    Answer in JSON format, with 'fy' key.

    Question: {question}

    Output JSON: {{
      "fy": <fiscal year in the question>
    }}
    """
)

from pydantic import BaseModel, Field

class FiscalYear(BaseModel):
    """FiscalYear mentioned in the given question"""
    fy: int = Field(description="fiscal year in the question")

extract_fiscal_chain = extract_fiscal_prompt | llm.with_structured_output(FiscalYear)

In [ ]:
from langchain_core.tools import tool

@tool
def rewrite2fiscal(question: str) -> tuple[str, int]:
  """Extract the company and convert relative time information in the question to the actual fiscal year of the company.
     It uses today function for the date information.

    Args:
        question: Question to be extracted.

    Returns:
        A tuple of ticker of the company and the fiscal year
  """
  ticker = extract_ticker_chain.invoke({"question": question}).ticker
  today = datetime.now().strftime("%Y-%m-%d")
  response = convert_chain.invoke({"original_question": question, "fiscal_info": get_fiscal_info_with_ticker(ticker), "today": today})
  fy = extract_fiscal_chain.invoke({'question': response}).fy
  return ticker, fy

In [ ]:
rewrite2fiscal("What was the revenue of Apple in 2023-07-10?")

In [ ]:
rewrite2fiscal("What was the revenue of Apple in 2023-12-20?")

In [ ]:
rewrite2fiscal("What was the revenue of Apple in last year?")

In [ ]:
rewrite2fiscal("What was the revenue of Apple in this year?")

### Domain specific calulation tools

You should implement more tools to answer complex questions.

In [ ]:
@tool
def calculate_eps(net_income: float, outstanding_shares: int):
  """Calculate the EPS of the company using net income and outstanding share

   Args:
        net_income: Net income value of the company
        outstanding_shares: Total stock held by the company's shareholders

    Returns:
        EPS value
  """
  if (net_income is None) or (outstanding_shares is None):
    print("Give me the net income and outstanding shares first")
    return None
  elif outstanding_shares == 0:
    print("Outstanding shares cannot be zero")
    return None

  return net_income / outstanding_shares

In [ ]:
@tool
def calculate_cashflowfromoperations(net_income: float, non_cash_items: float, changes_in_working_capital: float):
  """Calculate the cash flow from operations of the company using net income, non cash items and change in working capital

   Args:
        net_income: Net income value of the company
        non_cash_items: Financial transactions or events that are recorded in a company's financial statements but do not involve the exchange of cash
        changes_in_working_capital: Difference in a company's working capital between two reporting periods

    Returns:
        Value of cash flow from operations
  """
  if (net_income is None) or (non_cash_items is None) or (changes_in_working_capital is None):
    print("Give me the net income, non cash items and change in working capital data first")
    return None

  return net_income + non_cash_items + changes_in_working_capital

### Retriever Tool

#### Build Vector DB

You can skip this part for demo.

In [ ]:
import json

def load_and_view_sample(file_path):
  """Loads a JSON file and prints a sample of its contents."""
  try:
    with open(file_path, 'r') as f:
      data = json.load(f)
      print("Sample Data from JSON:")
      print(json.dumps(data, indent=2)[:500]) # Print first 500 chars for sample
  except FileNotFoundError:
    print(f"File not found: {file_path}")
  except json.JSONDecodeError:
    print(f"Error decoding JSON from file: {file_path}")


# Assuming your file is named 'test.json' in the current directory
load_and_view_sample('test.json')

In [ ]:
import json

def count_rows_in_json(file_path):
  """Counts the number of rows (objects) in a JSON file."""
  try:
    with open(file_path, 'r') as f:
      data = json.load(f)
      if isinstance(data, list):
        return len(data)
      elif isinstance(data, dict):
        return len(data)
      else:
        return 0  # Or raise an error, depending on your needs
  except FileNotFoundError:
    print(f"File not found: {file_path}")
    return 0
  except json.JSONDecodeError:
    print(f"Error decoding JSON from file: {file_path}")
    return 0

# Assuming your file is named 'test.json' in the current directory
num_rows = count_rows_in_json('test.json')
print(f"Number of rows in the JSON file: {num_rows}")

In [ ]:
import json

def analyze_json(file_path):
  """Analyzes a JSON file, printing information about its attributes."""
  try:
    with open(file_path, 'r') as f:
      data = json.load(f)
      if isinstance(data, list):
        print("JSON file contains a list of objects.")
        if len(data) > 0:
          print("Example object:")
          print(json.dumps(data[0], indent=2))
          for key in data[0].keys():
            print(f"- Attribute: {key}")
      elif isinstance(data, dict):
        print("JSON file contains a dictionary.")
        print("Attributes:")
        for key in data.keys():
          print(f"- Attribute: {key}")
      else:
        print("JSON file is empty or contains an unexpected data type.")
  except FileNotFoundError:
    print(f"File not found: {file_path}")
  except json.JSONDecodeError:
    print(f"Error decoding JSON from file: {file_path}")

# Analyze 'test.json'
analyze_json('test.json')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime
import json

# Define pre_text and post_text
pre_text = """
## Financial Data Analysis with Tools

This system is designed to analyze financial data and answer your questions about companies.
It uses tools like calculating metrics (EPS, cash flow from operations) and retrieving information from a financial dataset.

"""

post_text = """
**Note:** The system may not always have the information you're looking for.
If the response is insufficient, please refine your question or provide more details.
"""

# Define a table with examples
table = """
| Question Example | Expected Outcome |
|---|---|
| What was Apple's EPS in 2022? | Retrieves Apple's 2022 net income and shares outstanding, then calculates and returns EPS. |
| What was the cash flow from operations for Microsoft in 2023? | Retrieves Microsoft's net income, non-cash items, and changes in working capital, then calculates and returns cash flow from operations. |
"""

# Combine pre_text, post_text, and table
combined_text = f"{pre_text}\n\n**Example Usage:**\n\n{table}\n\n{post_text}"

# Function to ask ChatGPT and compare answers
def qa_and_compare(question):
    prompt_with_context = f"{combined_text}\n\n**Question:** {question}"

    messages = [HumanMessage(content=prompt_with_context)]

    ai_msg = llm.invoke(messages)

    print("ChatGPT's Answer:")
    print(ai_msg.content)


# Example question
question = "What was the revenue of Apple in 2022?"

qa_and_compare(question)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

embeddings = OpenAIEmbeddings(model='text-embedding-3-small',api_key=OPENAI_API_KEY)

# Assuming 'distinct_items_with_context' is a dictionary as shown in your code
for filename, context_data in distinct_items_with_context.items():
    pre_text = merge_text_list(context_data['pre_text'])
    post_text = merge_text_list(context_data['post_text'])
    table = context_data['table']

    # Split the pre-text and post-text into smaller chunks
    pre_text_chunks = splitter.split_text(pre_text)
    post_text_chunks = splitter.split_text(post_text)

    # Create a list of texts to store in the vector DB
    texts_for_db = []
    metadatas_for_db = []

    # Add pre-text chunks
    for chunk in pre_text_chunks:
        texts_for_db.append(chunk)
        metadatas_for_db.append({'company': filename.split('/')[0], 'fiscal': int(filename.split('/')[1]), 'context_type': 'pre_text'})

    # Add table
    if table:
        texts_for_db.append(table)
        metadatas_for_db.append({'company': filename.split('/')[0], 'fiscal': int(filename.split('/')[1]), 'context_type': 'table'})

    # Add post-text chunks
    for chunk in post_text_chunks:
        texts_for_db.append(chunk)
        metadatas_for_db.append({'company': filename.split('/')[0], 'fiscal': int(filename.split('/')[1]), 'context_type': 'post_text'})


    # Create the vector database for each file
    docsearch = Chroma.from_texts(texts_for_db, embeddings, metadatas=metadatas_for_db, persist_directory="./db")

In [ ]:
docsearch.persist()

In [ ]:
!zip -r ./db.zip ./db

In [ ]:
query = "What was the revenue of Apple in 2012?"
docs = docsearch.similarity_search(query)

for doc in docs:
  print(doc.page_content)
  print(doc.metadata)
  print('--------------')

#### Load Vector DB

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings(model='text-embedding-3-small',api_key=OPENAI_API_KEY)

from langchain_chroma import Chroma

docsearch = Chroma(
    persist_directory="./db",
    embedding_function=embeddings
)

#### Retriever tool with filtered search

In [ ]:
def retrieve(question:str, ticker: str, fy: int) -> list[str]:
  """Search vector DB for the financial reports with the question and ticker and fiscal year

   Args:
        question: Question need to be answered
        ticker: Ticker of the company for filtering the documents
        fy: Fiscal year for filtering the documents

    Returns:
        A related document for the question.
  """
  retriever = docsearch.as_retriever(search_kwargs={'k': 1, 'filter':
  {
      "$and": [
          {
              "company": {
                  "$eq": ticker
              }
          },
          {
              "fiscal": {
                  "$eq": fy
              }
          }
      ]
  }})
  result = retriever.invoke(question)
  if result:
    return result
  else:
    return ["No data returned. Try again with correct ticker and fiscal year, or different question"]

In [ ]:
retrieve("what was the hqla in the q4 of Citigroup in 2015?","AAPL", 2012)

In [ ]:
@tool
def retrieve_factual_data(question:str, ticker: str, fy: int) -> str:
  """Search vector DB for the financial reports with the question and ticker and fiscal year

   Args:
        question: Question need to be answered
        ticker: Ticker of the company for filtering the documents
        fy: Fiscal year for filtering the documents

    Returns:
        A related document for the question.
  """
  retriever = docsearch.as_retriever(search_kwargs={'k': 1, 'filter':
  {
      "$and": [
          {
              "company": {
                  "$eq": ticker
              }
          },
          {
              "fiscal": {
                  "$eq": fy
              }
          }
      ]
  }})
  result = retriever.invoke(question)
  if result:
    return result[0].page_content
  else:
    return "No data returned. Try again with correct ticker and fiscal year, or different question"

## Agent with tools

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

def agentic_rag(query: str) -> str:
  tools = [rewrite2fiscal, calculate_eps, calculate_cashflowfromoperations, retrieve_factual_data]
  system_query = 'You are the professional agent that can use tools to answer the financial question. Output answer without selecting tool if you can answer the question with chat history.'
  user_query = """For the given question, you should decide which tool you should use and arguments for the tool to answer the question.
        Question: {question}"""
  messages = [SystemMessage(content=system_query), HumanMessage(content=user_query.format(question=query))]
  llm_with_tools = llm.bind_tools(tools)
  ai_msg = llm_with_tools.invoke(messages)
  # print(ai_msg.content)
  while ai_msg.content == '':
    messages.append(ai_msg)
    for tool_call in ai_msg.tool_calls:
      selected_tool = {"rewrite2fiscal": rewrite2fiscal,
                      "calculate_eps": calculate_eps,
                      "calculate_cashflowfromoperations": calculate_cashflowfromoperations,
                      "retrieve_factual_data": retrieve_factual_data}[tool_call["name"].lower()]
      tool_msg = selected_tool.invoke(tool_call)
      messages.append(tool_msg)
    ai_msg = llm_with_tools.invoke(messages)
    # print(ai_msg.content)

  # for message in messages:
  #   print(message)
  return ai_msg.content

In [ ]:
response = agentic_rag("what was brazilian paper sales of International Paper Company in 2006?")

In [ ]:
response

## Scoring section

Use this Q&A dictionary to test your project code.

You should not change the scoring prompt.

You can submit your accuracy result with the screenshot to TA email, then I will post the score to the leaderboard.

In [ ]:
qa_dict = [{'Question': '"what was the hqla in the q4 of Citigroup in 2015"',
  'Answer': '"378.5 billion dollars"'},
 {'Question': '"as a result of the sales of certain non-core towers and other assets what was the recorded net losses of American Tower in 2008"',
  'Answer': '"10.5 million dollars"'},
 {'Question': '"what was the port call costsin of Royal Caribbean Cruises in 2012"',
  'Answer': '"459.8 million dollars"'},
 {'Question': '"what is the aggregate rent expense of American Tower Corp in 2014?"',
  'Answer': '"655.0 million dollars"'},
 {'Question': '"what is the long-term component of BlackRock at 12/31/2011?"',
  'Answer': '"593356 million dollars"'},
 {'Question': '"at december 31 , 2012 of The PNC Financial Services, what was the potential maximum exposure under the loss share arrangements?"',
  'Answer': '"3.9 billion dollars"'},
 {'Question': '"what is the total financial liabilities at fair value of Goldman Sachs in 2012?"',
  'Answer': '"377677 million dollars"'},
 {'Question': '"what was brazilian paper sales of International Paper Company in 2006?"',
  'Answer': '"6930 million dollars"'},
 {'Question': '"what was brazilian paper sales of International Paper Company in 2005?"',
  'Answer': '"7170 million dollars"'},
 {'Question': '"what is the cashflow of Kellogg in 2006?"',
  'Answer': '"957.4 million dollars"'},
 {'Question': '"From the perspective of 5 years ago, what percentage of total minimum lease payments of Dish Network are due in 2015?"',
  'Answer': '"10%"'},
 {'Question': '"in 2011 what was the SL Green Realty Corp\'s percent of the change in the account balance at end of year"',
  'Answer': '"93.7%"'},
 {'Question': '"what was the value in thousands of unvested restricted stock and performance awards at the weighted-averagegrant-datefair value as of december 31 , 2018 of Global Payments?"',
  'Answer': '"117624.84"'},
 {'Question': '"based on the cash dividends paid of the year, how many common stock shares were outstanding in 2007 Snap-on?"',
  'Answer': '"58909091"'},
 {'Question': '"what is the Entergy Corp\'s 2008 total value , in millions of dollars , of issuable long-term securities?"',
  'Answer': '"1450"'},
 {'Question': '"for the capital framework of Goldman Sachs Group 7 years ago, what percent of the minimum supplementary leverage ratio consisted of a buffer?"',
  'Answer': '"40%"'},
 {'Question': '"in 2007 what was the 3M\'s ratio of the interest expense to the interest income"',
  'Answer': '"1.59"'},
 {'Question': '"what is the AON\'s decrease observed in the additions for tax positions of prior years as of 2018, in millions?"',
  'Answer': '"2"'},
 {'Question': '"In 2014 Global Payments, what is the total value of securities approved by security holders but net yer issued , ( in millions ) ?"',
  'Answer': '"365.4"'},
 {'Question': '"what is the percent of our network route miles that is owned rather than operated on pursuant to trackage rights or leases in 2016 Union Pacific Corp"',
  'Answer': '"81.2%"'},
 {'Question': '"What is the American Tower Corp\'s current ratio in 12 years ago?"',
  'Answer': 1.7923},
 {'Question': '"What is the DISH Network Corporation\'s current ratio in 13 years ago?"',
  'Answer': 1.7804},
 {'Question': '"What is the American Tower Corp\'s current ratio in 2012?"',
  'Answer': 1.7923},
 {'Question': '"What is the DISH Network Corporation\'s current ratio in 2011?"',
  'Answer': 1.7804},
 {'Question': '"What is the International Paper Company\'s Operating Profit Margin in 2006?"',
  'Answer': 9.769119769},
 {'Question': '"What is the International Paper Company\'s Operating Profit Margin in 2005?"',
  'Answer': 6.59693166},
 {'Question': '"What is the International Paper Company\'s Operating Profit Margin in 2004?"',
  'Answer': 7.119831815},
 {'Question': '"What is the Air Products and Chemicals\'s Operating Profit Margin in 2016?"',
  'Answer': 26.7735375},
 {'Question': '"What is the Air Products and Chemicals\'s Operating Profit Margin in 2015?"',
  'Answer': 21.88472888},
 {'Question': '"What is the Air Products and Chemicals\'s Operating Profit Margin in 2014?"',
  'Answer': 18.69805075},
 {'Question': '"What is the difference of Operating Profit Margin between Air Products and Chemicals in 2016 and International Paper Company in 2006?"',
  'Answer': '17.00441774'},
 {'Question': '"What is the average of Operating Profit Margin between Air Products and Chemicals in 2016 and International Paper Company in 2005?"',
  'Answer': '16.68523458'},
 {'Question': '"What is the sum of Operating Profit Margin between Air Products and Chemicals in 2016 and International Paper Company in 2004?"',
  'Answer': '33.89336932'},
 {'Question': '"Which one\'s Operating Profit Margin is bigger between Air Products and Chemicals in 2014 and International Paper Company in 2006?"',
  'Answer': 'Air Products and Chemicals'},
 {'Question': '"What is the difference of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2006?"',
  'Answer': '12.11560911'},
 {'Question': '"What is the average of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2005?"',
  'Answer': '14.24083027'},
 {'Question': '"What is the sum of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2004?"',
  'Answer': '29.00456069'},
 {'Question': '"Which one\'s current ratio is bigger between American Tower Corp in 2012 and DISH Network Corporation in 2011?"',
  'Answer': 'American Tower Corp'},
 {'Question': '"What is the average of current ratio between American Tower Corp in 2012 and DISH Network Corporation in 2011?"',
  'Answer': '1.786341177'},
 {'Question': '"What is the difference of current ratio between American Tower Corp in 2012 and DISH Network Corporation in a year before that?"',
  'Answer': '0.01185521735'},
 {'Question': '"What is the Goldman Sachs Group\'s Total Assets change from 2014 to 2016 multiplied by the average of current ratio between American Tower Corp in 2012 and DISH Network Corporation in 2011?"',
  'Answer': '48722.45 million'},
 {'Question': '"What is the Goldman Sachs Group\'s Total Assets change from 2013 to 2017 multiplied by the average of current ratio between American Tower Corp in 2012 and DISH Network Corporation in 2011?"',
  'Answer': '46296.6 million'},
 {'Question': '"What is the sum of Goldman Sachs Group\'s Total Assets from 2011 to 2014 divided by the average of Operating Profit Margin between Air Products and Chemicals in 2016 and International Paper Company in 2005?',
  'Answer': '217529.28 million'},
 {'Question': '"What is the sum of Goldman Sachs Group\'s Total Assets from 2011 to 2014 multiplied by the average of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2005?"',
  'Answer': '51687477.96 million'},
 {'Question': '"What is the Goldman Sachs Group\'s Total Assets change from 2014 to 2016 divided the difference of Operating Profit Margin between Air Products and Chemicals in 2016 and International Paper Company in 2006?"',
  'Answer': '1604 million'},
 {'Question': '"What is the Goldman Sachs Group\'s Total Assets change from 2013 to 2017 dividied the sum of Operating Profit Margin between Air Products and Chemicals in 2016 and International Paper Company in 2004?"',
  'Answer': '764.6 million'},
 {'Question': '"What is the difference of Union Pacific Corp\'s revenue from 2013 to 2016 divided the sum of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2004?"',
  'Answer': '-69.7 million'},
 {'Question': '"What is the average of Union Pacific Corp\'s revenue from 2013 to 2016 divided by the sum of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2004?"',
  'Answer': '755.95 million'},
 {'Question': '"What is the difference of Union Pacific Corp\'s revenue from 2013 to 2016 multiplied by the average of current ratio between American Tower Corp in 2012 and DISH Network Corporation in 2011?"',
  'Answer': '-3611.98 million'},
 {'Question': '"What is the average of Union Pacific Corp\'s revenue from 2013 to 2016 divided by the average of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2005?"',
  'Answer': '1539.67 million'}]

In [ ]:
score_answer_prompt = PromptTemplate(
    input_variables=["question", "answer", "response"],
    template="""
    You have to score the response by comparing with the answer.
    You should score 0 or 1 as JSON with "score" key.
    You can ignore minor difference with the unit or numerical value.
    Question: {question}
    Answer: {answer}
    Response: {response}
    Score:

    Output JSON: {{
      "score": <1 if the answer response is meaningfully the same as the answer, 0 if the response is different from the answer>
    }}
    """
)

from pydantic import BaseModel, Field

class Score(BaseModel):
    """Score of the response"""
    score: int = Field(description="score of the response")

score_answer_chain = score_answer_prompt | llm.with_structured_output(Score)

In [ ]:
score_answer_chain.invoke({'question': qa_dict[0]['Question'], 'answer': qa_dict[0]['Answer'], 'response': 'it is 375 B'}).score

### Accuracy score

In [ ]:
correct = 0
for i, item in enumerate(qa_dict):
  response = agentic_rag(item['Question'])
  correct += score_answer_chain.invoke({'question': item['Question'], 'answer': item['Answer'], 'response': response}).score

print(f"Accuracy: {correct}/{len(qa_dict)}")